<a href="https://colab.research.google.com/github/Mondin0/data-eng/blob/main/CEL_ETL_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL - Luchtmeetnet API

In [ ]:
!pip install requests
!pip install deltalake
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 10.5 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import pyarrow as pa
from deltalake import write_deltalake, DeltaTable
from deltalake.exceptions import TableNotFoundError
from datetime import datetime, timedelta

In [ ]:
def get_data(base_url, endpoint, data_field=None, params=None, headers=None):
    """
    Realiza una solicitud GET a una API para obtener datos.

    Parámetros:
    base_url (str): La URL base de la API.
    endpoint (str): El endpoint de la API al que se realizará la solicitud.
    data_field (str): Atribudo del json de respuesta donde estará la lista
    de objetos con los datos que requerimos
    params (dict): Parámetros de consulta para enviar con la solicitud.
    headers (dict): Encabezados para enviar con la solicitud.

    Retorna:
    dict: Los datos obtenidos de la API en formato JSON.
    """
    try:
        endpoint_url = f"{base_url}/{endpoint}"
        response = requests.get(endpoint_url, params=params, headers=headers)
        response.raise_for_status()  # Levanta una excepción si hay un error en la respuesta HTTP.

        # Verificar si los datos están en formato JSON.
        try:
            data = response.json()
            if data_field:
              data = data[data_field]
        except:
            print("El formato de respuesta no es el esperado")
            return None
        return data

    except requests.exceptions.RequestException as e:
        # Capturar cualquier error de solicitud, como errores HTTP.
        print(f"La petición ha fallado. Código de error : {e}")
        return None

def build_table(json_data, record_path=None):
    """
    Construye un DataFrame de pandas a partir de datos en formato JSON.

    Parámetros:
    json_data (dict): Los datos en formato JSON obtenidos de una API.

    Retorna:
    DataFrame: Un DataFrame de pandas que contiene los datos.
    """
    try:
        df = pd.json_normalize(
            json_data,
            record_path)
        return df
    except:
        print("Los datos no están en el formato esperado")
        return None

def save_data_as_delta(df, path, mode="overwrite", partition_cols=None):
    """
    Guarda un dataframe en formato Delta Lake en la ruta especificada.
    A su vez, es capaz de particionar el dataframe por una o varias columnas.
    Por defecto, el modo de guardado es "overwrite".

    Args:
      df (pd.DataFrame): El dataframe a guardar.
      path (str): La ruta donde se guardará el dataframe en formato Delta Lake.
      mode (str): El modo de guardado. Son los modos que soporta la libreria
      deltalake: "overwrite", "append", "error", "ignore".
      partition_cols (list or str): La/s columna/s por las que se particionará el
      dataframe. Si no se especifica, no se particionará.
    """
    write_deltalake(
        path, df, mode=mode, partition_by=partition_cols
    )

def save_new_data_as_delta(new_data, data_path, predicate, partition_cols=None):
    """
    Guarda solo nuevos datos en formato Delta Lake usando la operación MERGE,
    comparando los datos ya cargados con los datos que se desean almacenar
    asegurando que no se guarden registros duplicados.

    Args:
      new_data (pd.DataFrame): Los datos que se desean guardar.
      data_path (str): La ruta donde se guardará el dataframe en formato Delta Lake.
      predicate (str): La condición de predicado para la operación MERGE.
    """

    try:
      dt = DeltaTable(data_path)
      new_data_pa = pa.Table.from_pandas(new_data)
      # Se insertan en target, datos de source que no existen en target
      dt.merge(
          source=new_data_pa,
          source_alias="source",
          target_alias="target",
          predicate=predicate
      ) \
      .when_not_matched_insert_all() \
      .execute()

    # Si no existe la tabla Delta Lake, se guarda como nueva
    except TableNotFoundError:
      save_data_as_delta(new_data, data_path, partition_cols=partition_cols)

def upsert_data_as_delta(data, data_path, predicate):
    """
    Guardar datos en formato Delta Lake usando la operacion MERGE.
    Cuando no haya registros coincidentes, se insertarán nuevos registros.
    Cuando haya registros coincidentes, se actualizarán los campos.

    Args:
      data (pd.DataFrame): Los datos que se desean guardar.
      data_path (str): La ruta donde se guardará el dataframe en formato Delta Lake.
      predicate (str): La condición de predicado para la operación MERGE.
    """
    try:
        dt = DeltaTable(data_path)
        data_pa = pa.Table.from_pandas(data)
        dt.merge(
            source=data_pa,
            source_alias="source",
            target_alias="target",
            predicate=predicate
        ) \
        .when_matched_update_all() \
        .when_not_matched_insert_all() \
        .execute()
    except TableNotFoundError:
        save_data_as_delta(data, data_path)

def read_most_recent_partition(data_path):
    """
    Lee la particion mas reciente de una tabla Delta Lake, teniendo en cuenta
    la fecha y hora actual.
    Supone que la tabla delta lake esta particionada por fecha y hora

    Args:
      data_path (str): La ruta donde se encuentra la tabla Delta Lake.

    Returns:
      pd.DataFrame: Los datos de la particion mas reciente.
    """
    try:
      requested_date = datetime.utcnow() - timedelta(hours=1)
      dt = DeltaTable(data_path)
      df_recent = dt.to_pandas(
        partitions=[
        ("fecha", "=", requested_date.strftime("%Y-%m-%d")),
        ("hora", "=", requested_date.strftime("%H"))
        ]
        )
      return df_recent
    except:
      raise Exception(f"No se pudo procesar la tabla Delta Lake, por {E}")
      return None

In [ ]:
base_url = "https://api.luchtmeetnet.nl/open_api"

## Extracción. Estaciones

In [ ]:
# Obtener todas las estaciones
endpoint = "stations"
params = {"organisation_id": 2}

stations = get_data(base_url, endpoint, data_field="data", params=params)
stations

[{'number': 'NL10248', 'location': 'Nistelrode-Gagelstraat'},
 {'number': 'NL10320', 'location': 'Burgh-Haamstede'},
 {'number': 'NL10404', 'location': 'Den Haag-Rebecquestraat'},
 {'number': 'NL10445', 'location': 'Den Haag-Amsterdamse Veerkade'},
 {'number': 'NL10246', 'location': 'Fijnaart-Zwingelspaansedijk'},
 {'number': 'NL10301', 'location': 'Zierikzee-Lange Slikweg'},
 {'number': 'NL10938', 'location': 'Groningen-Nijensteinheerd'},
 {'number': 'NL10107', 'location': 'Posterholt-Vlodropperweg'},
 {'number': 'NL10722', 'location': 'Eibergen-Lintveldseweg'},
 {'number': 'NL10929', 'location': 'Valthermond-Noorderdiep'},
 {'number': 'NL10617', 'location': 'Biddinghuizen-Kuilweg'},
 {'number': 'NL10538', 'location': 'Wieringerwerf-Medemblikkerweg'},
 {'number': 'NL10741', 'location': 'Nijmegen-Graafseweg'},
 {'number': 'NL10738', 'location': 'Wekerom-Riemterdijk'},
 {'number': 'NL10138', 'location': 'Heerlen-Jamboreepad'},
 {'number': 'NL10133', 'location': 'Wijnandsrade-Opfergeltst

In [ ]:
# Obtener detalles de cada station
all_stations = []

for station in stations:
  endpoint = f"stations/{station['number']}"

  station_details = get_data(base_url, endpoint, "data")
  if station_details:
    station_details["number"] = station["number"]
    station_details["location"] = station["location"]
    station_details.pop("province", None)
    all_stations.append(station_details)

In [ ]:
df_stations = build_table(all_stations)

In [ ]:
df_stations.head()

,type,components,municipality,url,organisation,location,year_start,number,geometry.type,geometry.coordinates,description.NL,description.EN
0,Regional,"[NO2, NO, PM10, NH3, LKI, PM25]",Bernheze,None,RIVM,Nistelrode-Gagelstraat,2022,NL10248,point,"[5.5433281, 51.69818779]",Nistelrode-Gagelstraat,Nistelrode
1,Regional,"[NH3, LKI]",Schouwen-Duiveland,None,RIVM,Burgh-Haamstede,2023,NL10320,point,"[3.7145, 51.70644]",Burgh-Haamstede,nieuw station
2,Municipal,"[NO, NO2, O3, PM10, PM25]",'s-Gravenhage,,RIVM,Den Haag-Rebecquestraat,,NL10404,point,"[4.289185, 52.077148]",Den Haag-Rebecquestraat,Den Haag-Rebecquestraat
3,Traffic,"[NO, PM10, NO2]",'s-Gravenhage,,RIVM,Den Haag-Amsterdamse Veerkade,,NL10445,point,"[4.315872, 52.075071]",Den Haag-Amsterdamse Veerkade,Den Haag-Amsterdamse Veerkade
4,Regional,"[NO, PM10, NO2]",Moerdijk,,RIVM,Fijnaart-Zwingelspaansedijk,,NL10246,point,"[4.515271, 51.653729]",Fijnaart-Zwingelspaansedijk,Fijnaart-Zwingelspaansedijk


## Almacenaniento. Estaciones

In [ ]:
bronze_dir = "lakehouse/bronze/luchtmeetnet_api"
stations_raw_dir = f"{bronze_dir}/stations"
save_new_data_as_delta(
    df_stations,
    stations_raw_dir,
    "target.number = source.number"
    )

## Extracción. Mediciones

In [ ]:
# Obtener mediciones

endpoint = "measurements"

start_date = datetime.utcnow() - timedelta(hours=2)
end_date = start_date.strftime("%Y-%m-%dT%H:59:59Z")
start_date = start_date.strftime("%Y-%m-%dT%H:00:00Z")

params = {
    "start": start_date,
    "end": end_date
    }

measurements = get_data(base_url, endpoint, params=params)
df_measurements = build_table(measurements, "data")

In [ ]:
df_measurements.sort_values("timestamp_measured", ascending=True)

,station_number,value,timestamp_measured,formula
0,NL01484,0.39,2025-03-18T22:00:00+00:00,C6H6
359,NL10449,34.14,2025-03-18T22:00:00+00:00,NOx
358,NL10449,33.16,2025-03-18T22:00:00+00:00,NO2
357,NL10449,0.59,2025-03-18T22:00:00+00:00,NO
356,NL10449,16.82,2025-03-18T22:00:00+00:00,PM25
...,...,...,...,...
168,NL01912,16.80,2025-03-18T22:00:00+00:00,NO
167,NL01494,20.10,2025-03-18T22:00:00+00:00,PM10
166,NL01494,8.60,2025-03-18T22:00:00+00:00,PM25
164,NL01494,0.88,2025-03-18T22:00:00+00:00,C6H6


## Almacenamiento. Mediciones

In [ ]:
df_measurements["timestamp_measured"] = pd.to_datetime(df_measurements.timestamp_measured)
df_measurements["fecha"] = df_measurements.timestamp_measured.dt.date
df_measurements["hora"] = df_measurements.timestamp_measured.dt.hour

measurements_raw_dir = f"{bronze_dir}/measurements"
save_new_data_as_delta(
    df_measurements,
    measurements_raw_dir,
    """target.timestamp_measured = source.timestamp_measured
    AND target.station_number = source.station_number
    AND target.formula = source.formula""",
    partition_cols=["fecha", "hora"]
    )

## Procesamiento

In [ ]:
df_stations_bronze = DeltaTable(stations_raw_dir).to_pandas()

df_measurements_bronze = read_most_recent_partition(measurements_raw_dir)

### Estaciones

In [ ]:
df_stations_bronze.head()

,type,components,municipality,url,organisation,location,year_start,number,geometry.type,geometry.coordinates,description.NL,description.EN
0,Regional,"[NO2, NO, PM10, NH3, LKI, PM25]",Bernheze,None,RIVM,Nistelrode-Gagelstraat,2022,NL10248,point,"[5.5433281, 51.69818779]",Nistelrode-Gagelstraat,Nistelrode
1,Regional,"[NH3, LKI]",Schouwen-Duiveland,None,RIVM,Burgh-Haamstede,2023,NL10320,point,"[3.7145, 51.70644]",Burgh-Haamstede,nieuw station
2,Municipal,"[NO, NO2, O3, PM10, PM25]",'s-Gravenhage,,RIVM,Den Haag-Rebecquestraat,,NL10404,point,"[4.289185, 52.077148]",Den Haag-Rebecquestraat,Den Haag-Rebecquestraat
3,Traffic,"[NO, PM10, NO2]",'s-Gravenhage,,RIVM,Den Haag-Amsterdamse Veerkade,,NL10445,point,"[4.315872, 52.075071]",Den Haag-Amsterdamse Veerkade,Den Haag-Amsterdamse Veerkade
4,Regional,"[NO, PM10, NO2]",Moerdijk,,RIVM,Fijnaart-Zwingelspaansedijk,,NL10246,point,"[4.515271, 51.653729]",Fijnaart-Zwingelspaansedijk,Fijnaart-Zwingelspaansedijk


In [ ]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   type                  25 non-null     object
 1   components            25 non-null     object
 2   municipality          25 non-null     object
 3   url                   23 non-null     object
 4   organisation          25 non-null     object
 5   location              25 non-null     object
 6   year_start            25 non-null     object
 7   number                25 non-null     object
 8   geometry.type         25 non-null     object
 9   geometry.coordinates  25 non-null     object
 10  description.NL        25 non-null     object
 11  description.EN        25 non-null     object
dtypes: object(12)
memory usage: 2.5+ KB


In [ ]:
# Tratamiento de nulos o datos vacíos
# 1. Eliminar columna totalmente vacia
df_stations_cleaned = df_stations_bronze.drop(columns=["url"])

# 2. Rellenar valores nulos
df_stations_cleaned["year_start"] = df_stations_cleaned["year_start"].replace("", 9999) # No hay valores, sino string vacios

In [ ]:
# 3. Crear columnas nuevas
df_stations_cleaned["geometry_coordinates_x"] = df_stations_cleaned["geometry.coordinates"].str[0]
df_stations_cleaned["geometry_coordinates_y"] = df_stations_cleaned["geometry.coordinates"].str[1]

In [ ]:
# 4. Renombrar columnas, reemplazando caracteres
df_stations_cleaned = df_stations_cleaned.rename(
                        columns=lambda col: col.replace(".", "_")
                        )

In [ ]:
df_stations_cleaned.head(1)

,type,components,municipality,organisation,location,year_start,number,geometry_type,geometry_coordinates,description_NL,description_EN,geometry_coordinates_x,geometry_coordinates_y
0,Regional,"[NO2, NO, PM10, NH3, LKI, PM25]",Bernheze,RIVM,Nistelrode-Gagelstraat,2022,NL10248,point,"[5.5433281, 51.69818779]",Nistelrode-Gagelstraat,Nistelrode,5.543328,51.698188


In [ ]:
# 5. Tipos de datos
type_mapping = {
    "type": "category",
    "municipality": "category",
    "organisation": "category",
    "year_start": "int16",
    "geometry_type": "category",
    "geometry_coordinates_x": "float32",
    "geometry_coordinates_y": "float32"
}

df_stations_cleaned = df_stations_cleaned.astype(type_mapping)

In [ ]:
stations_silver_dir = "lakehouse/silver/luchtmeetnet_api/stations"
save_new_data_as_delta(
    df_stations_cleaned,
    stations_silver_dir,
    "target.number = source.number"
    )

### Mediciones

In [ ]:
df_measurements_bronze.head()

,station_number,value,timestamp_measured,formula,fecha,hora
0,NL01489,11.7,2025-03-18 23:00:00+00:00,PM25,2025-03-18,23
1,NL01489,23.8,2025-03-18 23:00:00+00:00,PM10,2025-03-18,23
2,NL01487,3.0,2025-03-18 23:00:00+00:00,NO,2025-03-18,23
3,NL01487,31.3,2025-03-18 23:00:00+00:00,NO2,2025-03-18,23
4,NL01487,11.9,2025-03-18 23:00:00+00:00,PM25,2025-03-18,23


In [ ]:
df_measurements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   station_number      444 non-null    object             
 1   value               444 non-null    float64            
 2   timestamp_measured  444 non-null    datetime64[ns, UTC]
 3   formula             444 non-null    object             
 4   fecha               444 non-null    object             
 5   hora                444 non-null    int32              
dtypes: datetime64[ns, UTC](1), float64(1), int32(1), object(3)
memory usage: 19.2+ KB


In [ ]:
# 1. Eliminar duplicados
df_measurements_cleaned = df_measurements_bronze.drop_duplicates(
    subset=["station_number", "formula", "timestamp_measured"]
    )

In [ ]:
# Casteo
type_mapping = {
    "formula": "category",
    "fecha": "datetime64[ns]",
    "hora": "int8",
    }

df_measurements_cleaned = df_measurements_cleaned.astype(type_mapping)

In [ ]:
df_measurements_full = pd.merge(
        df_measurements_cleaned,
        df_stations_cleaned,
        how="inner",
        # on="column_name"
        left_on="station_number",
        right_on="number"
    )

In [ ]:
df_measurements_full.head()

,station_number,value,timestamp_measured,formula,fecha,hora,type,components,municipality,organisation,location,year_start,number,geometry_type,geometry_coordinates,description_NL,description_EN,geometry_coordinates_x,geometry_coordinates_y
0,NL10107,17.61,2025-03-18 23:00:00+00:00,NOx,2025-03-18,23,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
1,NL10107,17.11,2025-03-18 23:00:00+00:00,NO2,2025-03-18,23,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
2,NL10107,0.30,2025-03-18 23:00:00+00:00,NO,2025-03-18,23,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
3,NL10107,42.36,2025-03-18 23:00:00+00:00,O3,2025-03-18,23,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
4,NL10131,36.22,2025-03-18 23:00:00+00:00,O3,2025-03-18,23,Regional,"[FN, NH3, NO, NO2, O3, PM10, PM25]",Venray,RIVM,Vredepeel-Vredeweg,9999,NL10131,point,"[5.85307, 51.54052]",Vredepeel-Vredeweg,Vredepeel-Vredeweg,5.853070,51.54052


In [ ]:
# Guardar en silver
measurements_silver_dir = "lakehouse/silver/luchtmeetnet_api/measurements"
save_new_data_as_delta(
    df_measurements_full,
    measurements_silver_dir,
    """target.timestamp_measured = source.timestamp_measured
    AND target.station_number = source.station_number
    AND target.formula = source.formula""",
    partition_cols=["fecha", "hora"]
)

## OPCIONAL. Agregaciones

In [ ]:
df_measurements_silver = DeltaTable(measurements_silver_dir).to_pandas()

In [ ]:
df_measurements_silver.head()

,station_number,value,timestamp_measured,formula,fecha,hora,type,components,municipality,organisation,location,year_start,number,geometry_type,geometry_coordinates,description_NL,description_EN,geometry_coordinates_x,geometry_coordinates_y
0,NL10107,4.88,2024-12-03 19:00:00+00:00,NO,2024-12-03,19,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
1,NL10107,25.15,2024-12-03 19:00:00+00:00,NO2,2024-12-03,19,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
2,NL10107,22.98,2024-12-03 19:00:00+00:00,O3,2024-12-03,19,Regional,"[NO, NO2, O3]",Roerdalen,RIVM,Posterholt-Vlodropperweg,9999,NL10107,point,"[6.042399, 51.11919]",Posterholt-Vlodropperweg,Posterholt-Vlodropperweg,6.042399,51.11919
3,NL10131,0.90,2024-12-03 19:00:00+00:00,FN,2024-12-03,19,Regional,"[FN, NH3, NO, NO2, O3, PM10, PM25]",Venray,RIVM,Vredepeel-Vredeweg,9999,NL10131,point,"[5.85307, 51.54052]",Vredepeel-Vredeweg,Vredepeel-Vredeweg,5.853070,51.54052
4,NL10131,16.68,2024-12-03 19:00:00+00:00,NH3,2024-12-03,19,Regional,"[FN, NH3, NO, NO2, O3, PM10, PM25]",Venray,RIVM,Vredepeel-Vredeweg,9999,NL10131,point,"[5.85307, 51.54052]",Vredepeel-Vredeweg,Vredepeel-Vredeweg,5.853070,51.54052


In [ ]:
#formula and region
pd.pivot_table(
    df_measurements_silver,
    values="value",
    index=["formula", "type"],
    aggfunc=["mean", "min", "max", "std"]
    )

mean    min    max        std
                       value  value  value      value
formula type                                         
FN      Municipal   1.520000   0.25   2.61   1.190252
        Regional    0.470000   0.04   0.90   0.608112
        Traffic     1.565000   1.03   2.10   0.756604
NH3     Regional    8.380000   0.45  18.55   8.575864
NO      Municipal  13.672000  -0.25  57.05  24.573883
        Regional    0.930714  -0.06   4.88   1.482376
        Traffic     9.914000   5.29  11.62   2.616396
NO2     Municipal  28.426000  12.05  57.69  19.083498
        Regional   14.902143   2.43  32.70  10.499476
        Traffic    34.848000  25.09  41.60   8.143698
O3      Municipal  29.786000   0.09  64.12  28.124196
        Regional   45.068333  12.72  69.64  19.023837
        Traffic    32.925000  15.79  50.06  24.232549
PM10    Municipal  16.590000   9.55  26.19   7.855766
        Regional   11.044167   3.15  22.35   4.936603
        Traffic    16.718000  10.83  22.35   4.206509
PM25    Municipal   6.177500   3.84  11.15   3.353668
        Regional    5.107500   0.78   8.08   3.341929
        Traffic     6.230000   6.23   6.23        NaN
SO2     Regional    0.010000  -0.24   0.26   0.353553